In [ ]:
def get_conn_postgres():
    import psycopg2
    username = "hkcloud"
    password = "hkcloud1"
    host = "localhost"
    port = 5432
    database = "palette"
    dsn = f'{host}:{port}/{database}'
    encoding = 'UTF-8'

    try:
        connection = psycopg2.connect(host=host, database=database, user=username, password=password)
    except Exception as ex:
        print('Could not connect to database:', ex)
        return ex

    print("SUCCESS: Connecting database")
    return connection


In [ ]:
conn = get_conn_postgres()

In [ ]:
def get_df(sql):
    conn = get_conn_postgres()
    import pandas as pd
    with conn.cursor() as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()
        cols = [x[0].lower() for x in cursor.description]
        print(cols)
        df = pd.DataFrame(rows, columns = cols)
    #     for c in df.columns:
    #         if df[c].dtype == object:
    #             print(df[c])
    #             df[c] = df[c].astype("string")

    #     print(df)
        return df


In [ ]:
query = f"update {table} set ({cols}) VALUES ({ (',%s'*len(df.columns))[1:] })" 

### oracle

In [18]:
import oracledb
import pandas as pd
def get_conn_ora():
    
    username = "DPKMAPP"
    password = "Kms12#$"
    host = "61.106.19.217"
    port = 33389
    database = "CDB1"
    dsn = f'{host}:{port}/{database}'
    encoding = 'UTF-8'


    try:
        connection = oracledb.connect(user=username, password=password, dsn=dsn)
    except Exception as ex:
        print('Could not connect to database:', ex)
        return ex

    print("SUCCESS: Connecting database succeeded")
    return connection

def get_kms_datadf_ora(sql):
    try:
        connection = get_conn_ora()
     
        with connection.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()
            cols = [x[0].lower() for x in cursor.description]
   
        df = pd.DataFrame(rows, columns = cols)
        for c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].astype("string")

        connection.close()

    except Exception as ex:
        err, = ex.args
        print('error code :', err)
#         print('error message :', err.message)
        return None
   
    if len(df) ==0:
        print('No data found')  
        return None
    return df

In [19]:
sql = 'select * from tbctrg02'
df = get_kms_datadf_ora(sql)
df

SUCCESS: Connecting database succeeded


,room_id,room_seq,user_id,vtr_db_id,query_sentence,request_time
0,20240731110029290348,0,<NA>,CW,GREENER쿠폰에 대해 설명해줘,2024-07-31 11:00:25.853050
1,20240731112918071664,0,<NA>,plt,딤담당 통화시 대응요령은?,2024-07-31 11:29:14.961153
2,20240731143109869752,0,<NA>,plt,야근교통비 지급기준은?,2024-07-31 14:31:07.432711
3,20240731143109869752,1,<NA>,plt,장기출장 주말 귀향시 지원항목,2024-07-31 14:31:53.277809
4,20240801092042854490,0,<NA>,plt,kms기능을 요약해줘,2024-08-01 09:20:33.580512
...,...,...,...,...,...,...
169,20240821145503,1,teleweb,CW,위블비즈 고개겐터,2024-08-21 14:55:03.000000
170,20240821145513,1,teleweb,CW,위블비즈 고개겐터에 대해 표로 정리해줘,2024-08-21 14:55:13.000000
171,20240821145924,1,teleweb,CW,위블비즈,2024-08-21 14:59:24.000000
172,20240821150541,1,teleweb,CC,ㅁㄴㅇ,2024-08-21 15:05:41.000000


In [ ]:
def insert_df_to_table(df, table, mode='insert', repl_cond=None):
        """
        Using cursor.executemany() to insert the dataframe
        """
        # Create a list of tupples from the dataframe values
        tuples = list(set([tuple(x) for x in df.to_numpy()]))
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        table = table.upper()
        cols = cols.upper()


        query = f"INSERT INTO {table} ({cols}) VALUES ({ (',%s'*len(df.columns))[1:] })" 
        print('insert_df_to_table = ',query)
        print('data = ',tuples)

       # replace이면 기존 데이터 삭제
        if mode=='replace':
            del_sql = f"""delete from {table} where {repl_cond}"""
            if repl_cond == None:
                return False
            
        try:
            conn = get_conn_dbms()
            
            with conn.cursor() as cur:
                if mode=='replace':
                    cur.execute(del_sql)
                if len(df) > 0:
                    cur.executemany(query, tuples)
                conn.commit()

            conn.close()

        except (Exception) as error:
            print("Error: %s" % error)
            conn.rollback()
            conn.close()
            return False    
        return True

In [ ]:
import pandas as pd
import datetime
tt = datetime.datetime.now()
# ROOM_ID,ROOM_SEQ,DOC_SEQ,CNTNT_KEY,TITLE,SECTION,PARAGRAPH,RESPONSE_TIME
ll = ['202411',0,0,'52184>2>11','title','section','para',tt]
df= pd.DataFrame(ll).T
df.columns= ['ROOM_ID','ROOM_SEQ','DOC_SEQ','CNTNT_KEY','TITLE','SECTION','PARAGRAPH','IT_PROCESSING']
tuples = list(set([tuple(x) for x in df.to_numpy()]))
cols = ','.join(list(df.columns))
table ='DPKMAPP.TBCTRG04'
cols = cols.upper()
query = f"INSERT INTO {table} ({cols}) VALUES ({ (',%s'*len(df.columns))[1:] })" 
query

In [ ]:
for i in range(len(df.columns)):
    s = s+ ':' +str(i+1) + ',' 
query = f"INSERT INTO {table} ({cols}) VALUES ( {s[:-1] })"
query

In [ ]:
conn = get_conn_ora()
with conn.cursor() as cur:
     cur.executemany(query, tuples)
#     cur.executemany(query1, df.values.tolist())

In [ ]:
'''아래 예시는 templatepptx 임'''

sql='select * from tbctrg05'
df_comments = get_kms_datadf_ora(sql)
# df_pr
sql=f'''select a.room_id,a.room_seq
                    ,a.vtr_db_id
                    ,b.response_time - a.request_time  as res_time
                    ,a.request_time, b.response_time
                    ,a.query_sentence
                    ,b.llm_response
                    ,c.ans_relevancy
                    ,c.ans_faithfulness 
                    ,c.cntxt_precision
                    ,c.comments
                FROM tbctrg02 a
               INNER JOIN tbctrg03 b
                  ON a.room_id  = b.room_id and a.room_seq = b.room_seq 
               LEFT OUTER JOIN tbctrg05 c
                  ON c.eval_id = (SELECT max(eval_id)
                                   FROM tbctrg05
                                  WHERE room_id = a.room_id
                                    AND  room_seq = a.room_seq)
                order by request_time desc  
    '''
df_chat = get_kms_datadf_ora(sql)


In [ ]:
df_chat = df_chat.merge(df_comments[['room_id','room_seq','ans_relevancy','ans_faithfulness','cntxt_precision','comments']], how='left',on = ['room_id','room_seq'])
df_chat['ans_relevancy'] = df_chat['ans_relevancy'].fillna(0).apply(lambda x: '☆'*int(x))
df_chat['ans_faithfulness'] = df_chat['ans_faithfulness'].fillna(0).apply(lambda x: '☆'*int(x))
df_chat['cntxt_precision'] = df_chat['cntxt_precision'].fillna(0).apply(lambda x: '☆'*int(x))
df_chat['request_date'] = df_chat['request_time'].dt.date
df_sum = df_chat.groupby(['request_date','vtr_db_id']).agg({'room_id':'count'}).unstack().fillna(0)
df_sum = df_sum['room_id'].reset_index()

#### template부분 종료

In [ ]:
from datetime import datetime
from datetime import date

today = datetime.now()
today.strftime('%Y%m%d%H%M%S%f')

In [ ]:
dic = {}
import datetime

dt = datetime.datetime(2020,1,1)
dic['name'] = '이순신1'
dic['birth_date'] = '2020-1-1'
dic['gender'] = '남'
dic['education'] ='석사'
dic['school_name'] ='명랑'
dic['department_name']= '대찹'
dic['graduate_date'] = '1597-4-1'
dic['tech_grade']='특급'
dic['kosa'] = 'an'
print(type(dic))

In [ ]:
lst = [dic]
lst.append(dic)
print(len(lst))
[ tuple(x.values()) for x in lst]


In [ ]:
certi_id = [22,22]
certi_name=['a','b']
certi_date=[1,2]
dic = {}
dic['id'] = certi_id
dic['certi_name'] = certi_name
dic['certi_date'] = certi_date
# dict(zip(certi_name,certi_date))
# list(zip(tuple(dic.values())))
li = list(zip(certi_id,certi_name,certi_date))

In [ ]:
conn = get_conn_postgres()
sql='select * from members.certificates '
df = get_df(sql)
df_ct = df

In [ ]:
def insert_to_table(schema, table, data, get_seq=None):
    print(data)
    cols = ','.join(list(data.keys()))
    vals = str(tuple(data.values()))
    if schema:
        sql = f"""insert into {schema}.{table} (  {cols}  ) values  {vals}"""
    else:
        sql = f"""insert into {table} (  {cols}  ) values  {vals}"""

    seq = 0
    if get_seq:
        seq_sql = f"""SELECT currval('{get_seq}')"""
        print(seq_sql)
        
    try:
        connection = get_conn_postgres()
        connection.autocommit = True

        with connection.cursor() as cursor:
            cursor.execute(sql)
#             cursor.execute("SELECT currval('members.profiles_id_seq')")
            if get_seq:
                cursor.execute(seq_sql)
                seq = cursor.fetchone()[0]
        
        connection.close()

    except Exception as ex:
        err, = ex.args
#         print('error code :', err.code)
        print('error message :', err.message)
        return False
   
    return seq

In [ ]:
schema = 'members'
table = 'profiles'
insert_to_table(schema, table, data = dic, get_seq='members.profiles_id_seq' )

In [ ]:
df = pd.read_excel('C:/Users/hkccloud01/Documents/이력서테스트1.xlsx')

In [ ]:
connection = get_conn_postgres()
df.to_sql(name='careers',schema='members',con=connection)

In [ ]:
# id컬럼을 맨 앞에 추가
df.insert(0,'id',19)
df.columns = ['id','project_name','start_date','end_date','customer','role','job','environment','tech_stack','company','etc']

In [ ]:
from sqlalchemy import create_engine

In [ ]:
import psycopg2
import pandas as pd

def insertIntoTable(df, table):
        """
        Using cursor.executemany() to insert the dataframe
        """
        # Create a list of tupples from the dataframe values
        tuples = list(set([tuple(x) for x in df.to_numpy()]))
        print(tuples)
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        print(cols)
        # SQL query to execute
#         query = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s)" % (
#             table, cols)
        query = f"INSERT INTO {table} ({cols}) VALUES ({ (',%s'*len(df.columns))[1:] })" 
        print(query)
    
        try:
            conn = get_conn_postgres()
            with conn.cursor() as cur:    
                cur.executemany(query, tuples)
                conn.commit()
                conn.close()

        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            conn.close()
            return False

In [ ]:
import requests
import pandas as pd

In [ ]:
''' 4. retrieval '''

code = 'CW'
query = '보상쿠폰 지급기준에 대해 알려줘'
url = f'http://hkcloudai.com:8018/api/v1_1/retrieve/retrieve/retrieval_non_streaming?code={code}&query={query}'
res = requests.get(url)
res_str = res.json()  ## dictionary로 넣어줄 필요가 있음
res_str['llm_response'][0]

In [ ]:
res_str['retrieval_docs']

In [ ]:
# res_str['llm_response']
# res_str['retrieval_docs']
docs_seq, key,  section,para =[], [], [], []
for i,rel_doc in enumerate(res_str.get('retrieval_docs')): #'meta_data'
#     print(rel_doc.get('doc'))
    docs_seq.append(i)
    key.append(rel_doc.get('metadata').get('primary_key'))
    section.append(rel_doc.get('metadata').get('section'))
    para.append(rel_doc.get('metadata').get('paragraph'))
       
df = pd.DataFrame([docs_seq, key, section, para]).transpose()
df.columns=['docs_seq','key','section','paragraph']
df.insert(0,'room_id','room')
df.insert(1,'room_seq',1)
df.insert(len(df.columns),'it_processing',2)
df

In [ ]:
dic02={}
dic02['room_id']=['ss']
dic02['room_seq']=[2]
dic02['room_seq1']=[2]

df02 = pd.DataFrame.from_dict(dic02)

df02

In [ ]:
df

### db조회

In [ ]:
from models import database as db

In [ ]:
sql =  f""" select a.room_id,a.room_seq
                    ,a.vtr_db_id
                    ,b.response_time - a.request_time  as res_time
                    ,a.request_time, b.response_time
                    ,a.query_sentence
                    ,b.llm_response
                from aikms.tbctrg02 a, aikms.tbctrg03 b
                where a.room_id  = b.room_id
                and a.room_seq = b.room_seq 
                """
df = db.get_kms_datadf(sql)

### 사용자 쿼리를 최적화하는 query tansformation
#### 랭체인 rewrite
https://jerry-ai.com/41 \
https://blog.langchain.dev/query-transformations \
https://github.com/langchain-ai/langchain/blob/master/cookbook/rewrite.ipynb?ref=blog.langchain.dev

### 텍스트분석
https://m.blog.naver.com/j7youngh/222861344172

In [ ]:
!pip install konlpy, eunjeon
!pip install c:\mecab\mecab_python-0.996_ko_0.9.2_msvc-cp38-cp38-win_amd64.whl
!pip install mecab-ko-dic-msvc,mecab-ko-msvc

In [ ]:
import re

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거    
    text = re.sub(pattern, '', text)
    pattern = '([a-zA-Z0-9]+)'   # 알파벳, 숫자 제거  
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    return text

#### konply라이브러리

In [ ]:
txt = '국외출장비 지급기준에 대해 설명해 주세요'
from konlpy.tag import Kkma
kkma = Kkma()
kkma.nouns(txt)

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()
komoran.nouns(txt)

In [ ]:
from eunjeon import Mecab
mecab = Mecab()
mecab.nouns(txt)

In [ ]:
from konlpy.tag import Okt
okt = Okt()
okt.nouns(txt)

#### konply.mecab

In [ ]:
# cleansing
txt = df_chat['query_sentence'].str.cat(sep='\n')
corpus = text_clean(txt)
corpus

In [ ]:
# from konlpy.tag import Mecab
# from konlpy.tag import Hannanum, Komoran, Kkma, Okt, Mecab
from eunjeon import Mecab

In [ ]:
# mecab = Mecab(r'C:\Users\hkccloud01\anaconda3\Lib\site-packages/mecab-ko-dic')
mecab = Mecab()
tokens = mecab.nouns(corpus)

In [ ]:
from collections import Counter

In [ ]:
nouns_counter = Counter(tokens)
top_nouns = dict(nouns_counter.most_common(30))    # 딕셔너리 형태로 상위 30개 저장
top_nouns

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.font_manager as fm
# matplotlib가 사용하는 폰트 경로 확인
fontpath = 'c:/Windows/Fonts/NGULIM.TTF'
# 폰트 경로를 전달해 폰트이름을 가져옴
font_name = fm.FontProperties(fname=fontpath, size=10).get_name() 
# matplotlib가 사용할 글꼴을 나눔바른고딕으로 변경
plt.rc('font', family=font_name)


In [ ]:
plt.figure(figsize=(12, 12))               # 이미지 사이즈를 설정하고 이미지 생성
y_height = range(0, len(top_nouns))       # y축 높이 지정
plt.barh(y_height, top_nouns.values())     # 수평막대 그리기
plt.title("영화 헌트 리뷰 키워드 수")      #  차트 제목 설정
plt.yticks(y_height, top_nouns.keys())     #  y축 틱에 label 붙이기
plt.show()

In [ ]:
!pip install wordcloud STOPWORDS

In [ ]:
# 워드 클라우드
from wordcloud import WordCloud
import stopwords
import matplotlib.pyplot as plt

# 배경색과 폰트지정
wc= WordCloud(width = 3000, height = 2000, random_state=1, 
              background_color='white', colormap='Set2', 
              collocations=False, stopwords = stopwords,
              font_path='c:/Windows/Fonts/NGULIM.TTF')

# 워드클아우드 생성
wc.generate_from_frequencies(nouns_counter)    # 빈도수가 구해진 전체 단어plt.

# pyplot을 이용해 시각화
plt.figure(figsize=(12, 12))
plt.imshow(wc)
plt.axis("off")
plt.show()

#### krwordrank 라이브러리

In [ ]:
# !pip install krwordrank
from krwordrank.word import KRWordRank
min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)


In [ ]:
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
texts = ['운행 중 타이어 펑크 비용은 보상되나요?']
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)


In [ ]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

In [ ]:
stopwords = {'영화', '관람객', '너무', '정말', '보고'}
passwords = {word:score for word, score in sorted(
    keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}

In [ ]:
passwords

In [ ]:
result_text = ''
texts = f'''타이어 공기압 경고등이 켜질 때, 다음과 같은 조치를 취할 수 있습니다'''
sentence_pos = okt.pos(texts, stem=True)
nouns = [word for word, pos in sentence_pos if pos == 'Noun']
result_text = ' '.join(nouns)
noun_single_text_list = []
noun_single_text_list.append(result_text)

In [ ]:
from krwordrank.word import KRWordRank
min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 20
texts = noun_single_text_list
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
        print('%8s:\t%.4f' % (word, r))

#### openai를 이용한 키워드 추출은 gpt/chatGPT-QA.ipynb 에 있음 : 위 방법보다는 품질 좋음